<a href="https://www.kaggle.com/code/drkaggle22/toxic-comments-classification-tf-idf-nltk?scriptVersionId=176721119" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
df_train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip', index_col='id')

df_test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip', index_col='id')

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df = pd.concat([df_train, df_test])

In [5]:
df_train['comment_text'] = df_train['comment_text'].str.lower()
df_test['comment_text'] = df_test['comment_text'].str.lower()

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   toxic          159571 non-null  int64 
 2   severe_toxic   159571 non-null  int64 
 3   obscene        159571 non-null  int64 
 4   threat         159571 non-null  int64 
 5   insult         159571 non-null  int64 
 6   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 9.7+ MB


In [7]:
import re

def remove_special_characters(text):
    text = re.sub(r'http\S+', ' ', text )
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\bhttps?://[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\d', ' ', text)  # Corrected line
    text= re.sub(r'[\u4e00-\u9fff]+', ' ', text)
    return text

df_train['comment_text'] = df_train['comment_text'].apply(remove_special_characters)
df_test['comment_text'] = df_test['comment_text'].apply(remove_special_characters)

In [8]:
df_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
000103f0d9cfb60f,d aww he matches this background colour i m se...,0,0,0,0,0,0
000113f07ec002fd,hey man i m really not trying to edit war it s...,0,0,0,0,0,0
0001b41b1c6bb37e,more i can t make any real suggestions on impr...,0,0,0,0,0,0
0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [9]:
X = df_train['comment_text']  # Assuming 'comment_text' contains your text data
y = df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

In [10]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import pandas as pd

# Preprocessing your text data (tokenization)
# Assuming X is your text data
X_tokenized = X.apply(lambda text: text.split())

# Splitting the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_tokenized, y, train_size=0.8, random_state=42)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)

# Vectorize your text data using Word2Vec model
def word_averaging(model, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in model.wv.key_to_index:
            mean.append(model.wv[word])
            all_words.add(model.wv.key_to_index[word])
            
    if not mean:
        return np.zeros(model.vector_size,)

    mean = np.array(mean).mean(axis=0)
    return mean
# Concatenate training and validation data to fit the vectorizer
X_concat = pd.concat([X_train, X_valid])

def word_vectorizer(model, corpus):
    return np.array([word_averaging(model, sentence) for sentence in corpus])

X_train_w2v = word_vectorizer(word2vec_model, X_train)
X_valid_w2v = word_vectorizer(word2vec_model, X_valid)

# Now you can proceed to split the labels accordingly
y_concat = pd.concat([y_train, y_valid])
y_train, y_valid = y_concat[:len(X_train)], y_concat[len(X_train):]


# Print the shapes of transformed data for confirmation
print("X_train_w2v shape:", X_train_w2v.shape)
print("X_valid_w2v shape:", X_valid_w2v.shape)

# Now you can proceed to split the labels accordingly


X_train_w2v shape: (127656, 100)
X_valid_w2v shape: (31915, 100)


In [11]:
x = X_train_w2v
val_x = X_valid_w2v

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer


# Initialize MultiLabelBinarizer with the classes
mlb = MultiLabelBinarizer()

# Transform target labels into binary form
y_train_binary = mlb.fit_transform(y_train)
y_valid_binary = mlb.transform(y_valid)

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [13]:
import numpy as np

# List of class labels
class_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Initialize empty arrays to store binary labels
y_train_binary = np.zeros((len(y_train), len(class_labels)), dtype=int)
y_valid_binary = np.zeros((len(y_valid), len(class_labels)), dtype=int)

# Manually create binary labels for each class
for i, label in enumerate(class_labels):
    y_train_binary[:, i] = y_train[label]
    y_valid_binary[:, i] = y_valid[label]

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [14]:
print("y_train_binary shape:", y_train_binary.shape)
print("y_valid_binary shape:", y_valid_binary.shape)


y_train_binary shape: (127656, 6)
y_valid_binary shape: (31915, 6)


In [15]:
from sklearn.neural_network import MLPClassifier
import numpy as np

epochs =100
# Define MLPClassifier with custom activation function
clf = MLPClassifier(hidden_layer_sizes=(100, 64, 32, 10, 6),
                    activation='relu',
                    solver='adam',
                    random_state=42,
                    verbose=True,
                    learning_rate_init=0.001,
                    max_iter=epochs,  # Set the maximum number of iterations
                    early_stopping=False  # Disable automatic early stopping
                    )

# Fit the classifier with the training data and binary target labels
clf.fit(X_train_w2v, y_train_binary)


Iteration 1, loss = 0.56884363
Iteration 2, loss = 0.39729423
Iteration 3, loss = 0.37570452
Iteration 4, loss = 0.36824100
Iteration 5, loss = 0.36217746
Iteration 6, loss = 0.35650724
Iteration 7, loss = 0.35300565
Iteration 8, loss = 0.35068188
Iteration 9, loss = 0.34644493
Iteration 10, loss = 0.34321620
Iteration 11, loss = 0.34090112
Iteration 12, loss = 0.33803319
Iteration 13, loss = 0.33563251
Iteration 14, loss = 0.33269723
Iteration 15, loss = 0.33035113
Iteration 16, loss = 0.32696696
Iteration 17, loss = 0.32641855
Iteration 18, loss = 0.32139567
Iteration 19, loss = 0.31875122
Iteration 20, loss = 0.31756441
Iteration 21, loss = 0.31417900
Iteration 22, loss = 0.31189430
Iteration 23, loss = 0.30853778
Iteration 24, loss = 0.30771827
Iteration 25, loss = 0.30390917
Iteration 26, loss = 0.30185946
Iteration 27, loss = 0.29920437
Iteration 28, loss = 0.29657679
Iteration 29, loss = 0.29494185
Iteration 30, loss = 0.29253265
Iteration 31, loss = 0.28954738
Iteration 32, los

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(100, 64, 32, 10, 6), max_iter=100,
              random_state=42, verbose=True)

In [16]:
X_test_tokens = df_test['comment_text'].apply(lambda text: text.split())

# Transform the test data using the fitted TF-IDF vectorizer
X_test_tfidf = word_vectorizer(word2vec_model, X_test_tokens)

# Print the shape of transformed test data for confirmation
print("X_test_tfidf shape:", X_test_tfidf.shape)

X_test_tfidf shape: (153164, 100)


In [17]:
# Assuming clf is the trained MLPClassifier model

# Predict probabilities for test data
y_test_prob = clf.predict_proba(X_test_tfidf)

# Print the shape of predicted probabilities for confirmation
print("Shape of predicted probabilities:", y_test_prob.shape)

# Now you have predicted probabilities for each class for the test data


Shape of predicted probabilities: (153164, 6)


In [18]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [19]:
# Convert the predicted probabilities to float
y_test_prob_float = y_test_prob.astype(float)

# Print the updated type of predicted probabilities for confirmation
print("Type of predicted probabilities:", y_test_prob_float.dtype)

# Now y_test_prob_float contains predicted probabilities for each class for each data point in the test data, stored as float numbers


Type of predicted probabilities: float64


In [20]:
import numpy as np

# Assuming y_test_prob_float is your array of predicted probabilities

# Set numpy printing options to suppress scientific notation
np.set_printoptions(suppress=True)

# Print the updated predicted probabilities
print(y_test_prob_float)


[[0.99763705 0.55435734 0.98122177 0.36836302 0.97272548 0.62758235]
 [0.00005967 0.         0.00000159 0.         0.00000021 0.        ]
 [0.00000002 0.         0.         0.         0.         0.        ]
 ...
 [0.00154685 0.00000147 0.00026903 0.00000017 0.00004455 0.00000022]
 [0.00074048 0.00000013 0.0000385  0.00000001 0.00001082 0.00000002]
 [0.49532179 0.00319072 0.06568048 0.01298109 0.08873149 0.00621202]]


In [21]:
# Set the floating-point format for display
pd.options.display.float_format = '{:.10f}'.format

# Create submission DataFrame
submid = pd.DataFrame({'id': df_test.index})  # Use index as 'id' column
submission = pd.concat([submid, pd.DataFrame(y_test_prob_float, columns=classes)], axis=1)

# Write submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)


In [22]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.9976370515,0.5543573408,0.9812217703,0.3683630173,0.9727254775,0.6275823475
1,0000247867823ef7,0.0000596729,0.0000000019,0.0000015901,0.0000000000,0.0000002106,0.0000000000
2,00013b17ad220c46,0.0000000190,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,00017563c3f7919a,0.0002855516,0.0000000263,0.0000115672,0.0000000008,0.0000024392,0.0000000018
4,00017695ad8997eb,0.0000318346,0.0000000007,0.0000007541,0.0000000000,0.0000000798,0.0000000000
